In [2]:
# Importing Packages and Data
import pandas as pd
import numpy as np
import biogeme as biogeme
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
import biogeme.messaging as msgnotes
from biogeme.expressions import Beta
from biogeme.expressions import (
    Beta,
    DefineVariable,
    bioDraws,
    PanelLikelihoodTrajectory,
    MonteCarlo,
    log,
    Derive,
    bioNormalCdf,
    Elem
)
# from biogeme.results import calculate_correlation
import math
from datetime import datetime
import matplotlib.pyplot as plt
from factor_analyzer import FactorAnalyzer

In [3]:
# Import psychometric variables
psych_vars = pd.read_csv('Data/psych_vars.csv')


In [4]:
psych_vars.isnull().sum().sum()

3

In [5]:

# drop NA values
psych_vars = psych_vars.dropna()
psych_vars.isnull().sum().sum()

0

In [6]:

# rename columns
psych_vars.rename(columns={'q16':'socialnorm1', 'q17':'socialnorm2', 'q18':'socialnorm3', 'q19':'socialnorm4', 'q20':'socialnorm5', 'q21':'socialnorm6'}, inplace=True)
psych_vars.rename(columns={'q22': 'expectedperf1', 'q23': 'expectedperf2', 'q24': 'expectedperf3', 'q25': 'expectedperf4', 'q26': 'expectedperf5'}, inplace=True)
psych_vars.rename(columns={'q27': 'envbenefit1', 'q28': 'envbenefit2', 'q29': 'envbenefit3', 'q30': 'envbenefit4'}, inplace=True)
psych_vars.rename(columns={'q31': 'perceivedrisk1', 'q32': 'perceivedrisk2', 'q33': 'perceivedrisk3'}, inplace=True)
psych_vars.rename(columns={'q34': 'resideffects1', 'q35': 'resideffects2', 'q36': 'resideffects3', 'q37': 'resideffects4'}, inplace=True)
psych_vars.rename(columns={'q38': 'envvalues1', 'q39': 'envvalues2', 'q40': 'envvalues3', 'q41': 'envvalues4'}, inplace=True)
psych_vars.rename(columns={'q42': 'antimicro1', 'q43': 'antimicro2', 'q44': 'antimicro3'}, inplace=True)
psych_vars.rename(columns={'q45': 'intentiontouse1', 'q46': 'intentiontouse2', 'q47': 'intentiontouse3'}, inplace=True)

# convert likert scale responses into numerical values with 6 being "positive" and "1" being negative
# socialnorm1 --> "my family or friends think using bikesharing or scootershariing is a positive thing"
psych_vars.replace({'socialnorm1': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# socialnorm 2 --> "people important to me think that using bikesharing or scootersharing is a positive thing"
psych_vars.replace({'socialnorm2': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# socialnorm3 --> "in the near future more people in my city will use bikesharing or scootersharing"
psych_vars.replace({'socialnorm3': {'Considerably fewer': 1, 'Fewer': 2, 'Slightly fewer': 3, 'Slightly more': 4, 'More': 5, 'A lot more': 6}}, inplace=True)
# socialnorm4 --> "people who are important to me think that I should use bikesharing or scooter sharing"
psych_vars.replace({"socialnorm4": {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# socialnorm5 --> "it is a shame to use bikeshaing or scootersharing"
psych_vars.replace({'socialnorm5': {'Strongly agree': 1, 'Agree': 2, 'Slightly agree': 3, 'Slightly disagree': 4, 'Disagree': 5, 'Strongly disagree': 6}}, inplace=True)

## this one has a NaN
# socialnorm6 --> "the social media evaluates bikesharing or scootersharing negatively"
psych_vars.replace({'socialnorm6': {'Very negative': 1, 'Negative': 2, 'Somewhat negative': 3, 'Somewhat positive': 4, 'Positive': 5, 'Very positive': 6}}, inplace=True)

# expectedperf1 --> "shared micromobility is convenient"
psych_vars.replace({'expectedperf1': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# expected perf2 --> "shared micromobility is effective for my personal mobility"
psych_vars.replace({'expectedperf2': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# expectedperf3 --> "shared micromobility can help me reach my destination efficiently"
psych_vars.replace({'expectedperf3': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# expectedperf4 --> "there are enough shared bikes/scooters available whenever I want to use them"
psych_vars.replace({'expectedperf4': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# expectedperf5 --> "I can comfortably take rides on a shared bike or scooter for my daily business"
psych_vars.replace({'expectedperf5': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# envbenefit1 --> "using shared micromobility will help alleviate traffic congestion"
psych_vars.replace({'envbenefit1': {'Very unlikely': 1, 'Unlikely': 2, 'Slightly unlikely': 3, 'Slightly likely': 4, 'Likely': 5, 'Very likely': 6}}, inplace=True)
# envbenefit2 --> "using shared micromobility will reduce carbon emission and air pollution"
psych_vars.replace({'envbenefit2': {'Very unlikely': 1, 'Unlikely': 2, 'Slightly unlikely': 3, 'Slightly likely': 4, 'Likely': 5, 'Very likely': 6}}, inplace=True)
# envbenefit3 --> "using a shared bike or scooter fits my environmental concerns"
psych_vars.replace({'envbenefit3': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)

# envbenefit4 --> "shared micromobility has a positive impact on urban traffic"
##### CHECK THIS ONE (do answers make sense?), also has a NaN
psych_vars.replace({'envbenefit4': {'Significantly increase': 1, 'Increase': 2, 'Slightly increase': 3, 'Slightly decrease': 4, 'Decrease': 5, 'Significantly decrease': 6}}, inplace=True)

# perceivedrisk1 --> "I would feel safe riding a shared bike or scooter in traffic"
psych_vars.replace({'perceivedrisk1': {'Very unsafe': 1, 'Unsafe': 2, 'Somewhat unsafe': 3, 'Somewhat safe': 4, 'Safe': 5, 'Very safe': 6}}, inplace=True)
# perceivedrisk2 --> "I think riding a shared bike or scooter is dangerous"
psych_vars.replace({'perceivedrisk2': {'Very dangerous': 1, 'Dangerous': 2, 'Somewhat dangerous': 3, 'Somewhat safe': 4, 'Safe': 5, 'Very safe': 6}}, inplace=True)
# perceivedrisk3 --> "I would feel nervous about having an accident when riding a shared bike or scooter"
psych_vars.replace({'perceivedrisk3': {'Strongly agree': 1, 'Agree': 2, 'Slightly agree': 3, 'Slightly disagree': 4, 'Disagree': 5, 'Strongly disagree': 6}}, inplace=True)

##### CHECK THIS ONE -- normalize?
# resideffects1 --> "I knew about bikesharing or scootersharing before"
psych_vars.replace({'resideffects1': {'No': 0, 'Yes': 1}}, inplace=True)

# resideffects2 --> "many people around me know about bikesharing or scootersharing"
psych_vars.replace({'resideffects2': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)

# resideffects3 --> "I have used bikesharing or scootersharing before"
psych_vars.replace({'resideffects3': {'No': 0, 'Yes': 1}}, inplace=True)

# resideffects4 --> "there are bikesharing or scootersharing available to me and I can use them regularly"
psych_vars.replace({'resideffects4': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# envvalues1 --> "I would like to do my part to reduce carbon emission and air pollution"
psych_vars.replace({'envvalues1': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# envvalues2 --> "I always consider how my transport choices affect the environment"
psych_vars.replace({'envvalues2': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# envvalues3 --> "I consider myself to be an environmentally conscious person"
psych_vars.replace({'envvalues3': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# envvalues4 --> "global warming is fake science"
psych_vars.replace({'envvalues4': {'Strongly agree': 1, 'Agree': 2, 'Slightly agree': 3, 'Slightly disagree': 4, 'Disagree': 5, 'Strongly disagree': 6}}, inplace=True)
# antimicro1 --> "shared micromobility is a very bad idea"
psych_vars.replace({'antimicro1': {'Strongly agree': 1, 'Agree': 2, 'Slightly agree': 3, 'Slightly disagree': 4, 'Disagree': 5, 'Strongly disagree': 6}}, inplace=True)
# antimicro2 --> "shared micromobility causes a lot of problems"
psych_vars.replace({'antimicro2': {'Strongly agree': 1, 'Agree': 2, 'Slightly agree': 3, 'Slightly disagree': 4, 'Disagree': 5, 'Strongly disagree': 6}}, inplace=True)
# antimicro3 --> "shared micromobility should not have existed in cities"
psych_vars.replace({'antimicro3': {'Strongly agree': 1, 'Agree': 2, 'Slightly agree': 3, 'Slightly disagree': 4, 'Disagree': 5, 'Strongly disagree': 6}}, inplace=True)
# intentiontouse1 --> "I'm willing to use bikesharing or scootersharing in the future"
psych_vars.replace({'intentiontouse1': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)
# intentiontouse2 --> "I would recommend friends and family to use bikesharing or scootersharing"

### naN in this one
psych_vars.replace({'intentiontouse2': {'Definetely would not': 1, 'Probably would not': 2, 'Maybe would not': 3, 'Maybe would': 4, 'Probably would': 5, 'Definetely would': 6}}, inplace=True)

# intentiontouse3 --> "I'm willing to use bikesharing or scootersharing on a regular basis"
psych_vars.replace({'intentiontouse3': {'Strongly disagree': 1, 'Disagree': 2, 'Slightly disagree': 3, 'Slightly agree': 4, 'Agree': 5, 'Strongly agree': 6}}, inplace=True)


In [7]:
# Import micromobility data
micro_pool_socio = pd.read_csv('Data/micro_pool_socio_bio2up.csv')

In [9]:
# create new dataframe with factor columns
factor_df_with_user = psych_vars[['user', 'socialnorm1', 'socialnorm2', 'socialnorm3', 'socialnorm4', 'socialnorm5', 'socialnorm6', 'expectedperf1', 'expectedperf2', 'expectedperf3', 'expectedperf4', 'expectedperf5', 'envbenefit1', 'envbenefit2', 'envbenefit3', 'envbenefit4', 'perceivedrisk1', 'perceivedrisk2', 'perceivedrisk3', 'resideffects1', 'resideffects2', 'resideffects3', 'resideffects4', 'envvalues1', 'envvalues2', 'envvalues3', 'envvalues4', 'antimicro1', 'antimicro2', 'antimicro3', 'intentiontouse1', 'intentiontouse2', 'intentiontouse3']]
factor_df = psych_vars[['socialnorm1', 'socialnorm2', 'socialnorm3', 'socialnorm4', 'socialnorm5', 'socialnorm6', 'expectedperf1', 'expectedperf2', 'expectedperf3', 'expectedperf4', 'expectedperf5', 'envbenefit1', 'envbenefit2', 'envbenefit3', 'envbenefit4', 'perceivedrisk1', 'perceivedrisk2', 'perceivedrisk3', 'resideffects1', 'resideffects2', 'resideffects3', 'resideffects4', 'envvalues1', 'envvalues2', 'envvalues3', 'envvalues4', 'antimicro1', 'antimicro2', 'antimicro3', 'intentiontouse1', 'intentiontouse2', 'intentiontouse3']]
print(factor_df.head())
fa = FactorAnalyzer(rotation='promax', n_factors=8, method = 'ml')
fa.fit(factor_df)
# Extract the 'user' column
user_column = factor_df_with_user['user']

# Calculate the factor scores
scores = fa.transform(factor_df)

# Create a new DataFrame with factor scores and 'user' column
factor_scores_df = pd.DataFrame(scores, columns=['Factor_1', 'Factor_2', 'Factor_3', 'Factor_4', 'Factor_5', 'Factor_6', 'Factor_7', 'Factor_8'])
factor_scores_df['user'] = user_column

print(factor_scores_df)

   socialnorm1  socialnorm2  socialnorm3  socialnorm4  socialnorm5  \
0            4            4            5            4            6   
1            5            5            4            5            6   
2            5            5            6            2            6   
3            4            4            5            3            6   
4            4            4            4            2            6   

   socialnorm6  expectedperf1  expectedperf2  expectedperf3  expectedperf4  \
0            4              5              4              4              1   
1            4              5              5              5              3   
2            6              6              2              2              4   
3            5              5              4              4              2   
4            4              5              1              2              2   

   ...  envvalues1  envvalues2  envvalues3  envvalues4  antimicro1  \
0  ...           5           5          

In [10]:
# merge factor_scores_df and psych_vars on 'user'
psych_vars_with_factors = pd.merge(psych_vars, factor_scores_df, on = 'user')

In [11]:
# Import scoot_user_id.csv
scoot_user_id = pd.read_csv('Data/scoot_user_id.csv')

# merge micro_pool_socio and scooter_user_id on 'p'
micro_pool_socio = pd.merge(micro_pool_socio, scoot_user_id, on='p')


In [12]:
# merge psych_vars and micro_pool_socio using inner join, thus dropping any users who did not answer psychometric questions
combined_psych = pd.merge(micro_pool_socio, psych_vars_with_factors, on='user', )

# get count of unique users
combined_psych['user'].nunique()

# sort by dperson
combined_psych = combined_psych.sort_values(by=['who'])

# drop user column b/c is it is type string
combined_psych = combined_psych.drop(columns=['user'])

# create new column called age_35_more
combined_psych['age_35_more'] = np.where(combined_psych['age'] >= 35, 1, 0)
# create new column for child > 0
combined_psych['have_children'] = np.where(combined_psych['child'] > 0, 1, 0)
# create new column for fulltime_employee where work = 1
combined_psych['fulltime_employee'] = np.where(combined_psych['work'] == 1, 1, 0)
# create new column for hh_income_75k where income >= 8
combined_psych['hh_income_75k'] = np.where(combined_psych['hhincome'] >= 8, 1, 0)
# # create new column for BKLN_Yes
# combined_psych['BKLN_Yes'] = np.where((combined_psych['BKLN'] == 1) | (combined_psych['BKLN'] == 2) | (combined_psych['BKLN'] == 3), 1, 0)
combined_psych['college_grad'] = np.where(combined_psych['edu'] >= 4, 1, 0)
combined_psych['PRCP_YES'] = np.where(combined_psych['PRCP'] > 1, 1, 0)
combined_psych['BKLN_YES'] = np.where(combined_psych['BKLN'] > 1, 1, 0)

# convert to database
database = db.Database('combined_psych', combined_psych)

database.panel("who")


globals().update(database.variables)

In [13]:
# print the number of unique entries in the "who" column of combined_psych
print(combined_psych['who'].nunique())


1748


In [14]:
# create a new dataframe with only the unique 'who' values in combined_psych
respondents = combined_psych.drop_duplicates(subset=['who']).copy()

total_rows = 1748

# Calculate statistics for 'age' variable
age_min = respondents['age'].min()
age_median = respondents['age'].median()
age_mean = respondents['age'].mean()
age_max = respondents['age'].max()

# Print the statistics
print(f"Min: {age_min}")
print(f"Median: {age_median}")
print(f"Mean: {age_mean:.2f}")
print(f"Max: {age_max}")

# Calculate percentages for each category manually
gender_F_percentage = (respondents['gender_F'].sum() / total_rows * 100)
gender_M_percentage = (respondents['gender_M'].sum() / total_rows) * 100
gender_O_percentage = (respondents['gender_O'].sum() / total_rows) * 100
race_white_percentage = (respondents['race'] == 1).sum() / total_rows * 100
race_black_percentage = (respondents['race'] == 2).sum() / total_rows * 100
race_american_indian_percentage = (respondents['race'] == 3).sum() / total_rows * 100
race_asian_percentage = (respondents['race'] == 4).sum() / total_rows * 100
race_pacific_percentage = (respondents['race'] == 5).sum() / total_rows * 100

education_less_than_bach = (respondents['edu'] < 5).sum() / total_rows * 100
education_bachelor_higher_percentage = (respondents['edu'] >= 5).sum() / total_rows * 100

employment_employed_percentage = (respondents['work'] <= 2).sum() / total_rows * 100
employment_not_employed_percentage = (respondents['work'] > 2).sum() / total_rows * 100

income_under_25k_percentage = (respondents['hhincome'] <= 4).sum() / total_rows * 100
income_25k_50k_percentage = ((respondents['hhincome'] == 5) | (respondents['hhincome'] == 6)).sum() / total_rows * 100
income_50k_75k_percentage = ((respondents['hhincome'] == 7)).sum() / total_rows * 100
income_75k_100k_percentage = (respondents['hhincome'] == 8).sum() / total_rows * 100
income_100k_150k_percentage = (respondents['hhincome'] == 9).sum() / total_rows * 100
income_above_150k_percentage = (respondents['hhincome'] >= 10).sum() / total_rows * 100


# car_0_percentage = (combined_psych['car'] == 0).sum() / total_rows * 100
# car_1_percentage = (combined_psych['car'] == 1).sum() / total_rows * 100
# car_2_percentage = (combined_psych['car'] == 2).sum() / total_rows * 100
# car_3_percentage = (combined_psych['car'] == 3).sum() / total_rows * 100
# car_above_4_percentage = (combined_psych['car'] >= 4).sum() / total_rows * 100

# Print the percentages
print(f"Female: {gender_F_percentage:.1f}%")
print(f"Male: {gender_M_percentage:.1f}%")
print(f"Another: {gender_O_percentage:.1f}%")
print(f"White: {race_white_percentage:.1f}%")
print(f"Black or African American: {race_black_percentage:.1f}%")
print(f"American Indian or Alaska Native: {race_american_indian_percentage:.1f}%")
print(f"Asian: {race_asian_percentage:.1f}%")
print(f"Native Hawaiian or Other Pacific Islander: {race_pacific_percentage:.1f}%")
print(f"Less than bachelor’s degree: {education_less_than_bach:.1f}%")
print(f"Bachelor’s degree or higher: {education_bachelor_higher_percentage:.1f}%")
print(f"Employed: {employment_employed_percentage:.1f}%")
print(f"Not employed: {employment_not_employed_percentage:.1f}%")
print(f"Under $25,000: {income_under_25k_percentage:.1f}%")
print(f"$25,000-$49,999: {income_25k_50k_percentage:.1f}%")
print(f"$50,000-$74,999: {income_50k_75k_percentage:.1f}%")
print(f"$75,000-$99,999: {income_75k_100k_percentage:.1f}%")
print(f"$100,000-$149,999: {income_100k_150k_percentage:.1f}%")
print(f"≥  $150,000: {income_above_150k_percentage:.1f}%")



Min: 18
Median: 35.0
Mean: 37.87
Max: 84
Female: 42.0%
Male: 57.5%
Another: 0.5%
White: 81.3%
Black or African American: 9.7%
American Indian or Alaska Native: 0.8%
Asian: 6.3%
Native Hawaiian or Other Pacific Islander: 0.1%
Less than bachelor’s degree: 27.5%
Bachelor’s degree or higher: 72.5%
Employed: 89.8%
Not employed: 10.2%
Under $25,000: 14.0%
$25,000-$49,999: 25.7%
$50,000-$74,999: 26.3%
$75,000-$99,999: 17.5%
$100,000-$149,999: 10.6%
≥  $150,000: 5.9%


In [15]:
ASC_CAR = Beta('ASC_CAR', 0, None, None, 1)
# ASC_CAR_S = Beta('ASC_CAR_S', 1, None, None, 0)
# ASC_CAR_RND = ASC_CAR + ASC_CAR_S * bioDraws('ASC_CAR_RND', 'NORMAL_ANTI')

ASC_TRANSIT = Beta('ASC_TRANSIT', 0, None, None, 0)
# ASC_TRANSIT_S = Beta('ASC_TRANSIT_S', 1, None, None, 0)
# ASC_TRANSIT_RND = ASC_TRANSIT + ASC_TRANSIT_S * bioDraws('ASC_TRANSIT_RND', 'NORMAL_ANTI')

ASC_RH = Beta('ASC_RH', 0, None, None, 0)
# ASC_RH_S = Beta('ASC_RH_S', 1, None, None, 0)
# ASC_RH_RND = ASC_RH + ASC_RH_S * bioDraws('ASC_RH_RND', 'NORMAL_ANTI')

# ASC_SCOOTER = Beta('ASC_SCOOTER', 0, None, None, 0)
# ASC_SCOOTER_S = Beta('ASC_SCOOTER_S', 1, None, None, 0)
# ASC_SCOOTER_RND = ASC_SCOOTER + ASC_SCOOTER_S * bioDraws('ASC_SCOOTER_RND', 'NORMAL_ANTI')

# ASC_DLBIKE = Beta('ASC_DLBIKE', 0, None, None, 0)
# ASC_DLBIKE_S = Beta('ASC_DLBIKE_S', 1, None, None, 0)
# ASC_DLBIKE_RND = ASC_DLBIKE + ASC_DLBIKE_S * bioDraws('ASC_DLBIKE_RND', 'NORMAL_ANTI')

# ASC_DKBIKE = Beta('ASC_DKBIKE', 0, None, None, 0)
# ASC_DKBIKE_S = Beta('ASC_DKBIKE_S', 1, None, None, 0)
# ASC_DKBIKE_RND = ASC_DKBIKE + ASC_DKBIKE_S * bioDraws('ASC_DKBIKE_RND', 'NORMAL_ANTI')

# ASC_SCTRANSIT = Beta('ASC_SCTRANSIT', 0, None, None, 0)
# ASC_SCTRANSIT_S = Beta('ASC_SCTRANSIT_S', 1, None, None, 0)
# ASC_SCTRANSIT_RND = ASC_SCTRANSIT + ASC_SCTRANSIT_S * bioDraws('ASC_SCTRANSIT_RND', 'NORMAL_ANTI')

B_CARTIME = Beta('B_CARTIME', 0, None, None, 0)
B_TRANSITTIME = Beta('B_TRANSITTIME', 0, None, None, 0)
B_RHTIME = Beta('B_RHTIME', 0, None, None, 0)
B_WALKTIME = Beta('B_WALKTIME', 0, None, None, 0)
B_PERSBIKETIME = Beta('B_PERSBIKETIME', 0, None, None, 0)

B_SCOOTERTIME = Beta('B_SCOOTERTIME', 0, None, None, 0)
B_BIKETIME = Beta('B_BIKETIME', 0, None, None, 0)
# B_DKBIKETIME = Beta('B_DKBIKETIME', 0, None, None, 0)
B_SCTRANSITTIME = Beta('B_SCTRANSITTIME', 0, None, None, 0)

B_ACCESS = Beta('B_ACCESS', 0, None, None, 0)
B_DROP =Beta('B_DROP', 0, None, None, 0)
B_WAITAV = Beta('B_WAITAV', 0, None, None, 0)
B_AV = Beta('B_AV', 0, None, None, 0)

B_COST = Beta('B_COST', 0, None, None, 0)
B_COST_PRIME = Beta('B_COST_PRIME', 0, None, None, 0)
B_costadj = Beta('B_costadj', 0, None, None, 0)

B_PRCP_Yes = Beta('B_PRCP_Yes', 0, None, None, 0)
B_OWNBIKE = Beta('B_OWNBIKE', 0, None, None, 0)
B_BIKELANE = Beta('B_BIKELANE', 0, None, None, 0)
B_AGE_SQ = Beta('B_AGE_SQ', 0, None, None, 0)
B_AGE_SCTRANSIT_SQ = Beta('B_AGE_SCTRANSIT_SQ', 0, None, None, 0)
B_GENDER_F = Beta('B_GENDER_F', 0, None, None, 0)

B_AGE_MORE_35 = Beta('B_AGE_MORE_35', 0, None, None, 0)
B_CHILD_IN_HH = Beta('B_CHILD_IN_HH', 0, None, None, 0)
B_IS_WHITE = Beta('B_IS_WHITE', 0, None, None, 0)
B_FULLTIME_EMPLOY = Beta('B_FULLTIME_EMPLOY', 0, None, None, 0)
B_HHINCOME_MORE_75k = Beta('B_HHINCOME_MORE_75k', 0, None, None, 0)

B_RISK_BIKE = Beta('B_RISK_BIKE', 0, None, None, 0)
B_RISK_SCTRANSIT = Beta('B_RISK_SCTRANSIT', 0, None, None, 0)
B_RISK_TRANSIT = Beta('B_RISK_TRANSIT', 0, None, None, 0)
B_RISK_RH = Beta('B_RISK_RH', 0, None, None, 0)
B_RISK_PERSBIKE = Beta('B_RISK_PERSBIKE', 0, None, None, 0)
B_RISK_WALK = Beta('B_RISK_WALK', 0, None, None, 0)
B_RISK_SCOOTER = Beta('B_RISK_SCOOTER', 0, None, None, 0)

B_RISK_MICROMOBILITY = Beta('B_RISK_MICROMOBILITY', 0, None, None, 0)
B_RISK_ACTIVETRANSP = Beta('B_RISK_ACTIVETRANSP', 0, None, None, 0)

B_NORM_MICROMOBILITY_SCOOTER = Beta('B_NORM_MICRO_SCOOT', 0, None, None, 0)
B_NORM_MICROMOBILITY_BIKE = Beta('B_NORM_MICRO_BIKE', 0, None, None, 0)

B_ENVVALUE_MICROMOBILITY_SCOOTER = Beta('B_ENVVALUE_MICRO_SCOOT', 0, None, None, 0)
B_ENVVALUE_MICROMOBILITY_BIKE = Beta('B_ENVVALUE_MICRO_BIKE', 0, None, None, 0)
B_ENVVALUE_MICROMOBILITY = Beta('B_ENVVALUE_MICROMOBILITY', 0, None, None, 0)

In [16]:
ASC_CAR = Beta('ASC_CAR', 0, None, None, 1)
ASC_CAR_S = Beta('ASC_CAR_S', 1, None, None, 0)
ASC_CAR_RND = ASC_CAR + ASC_CAR_S * bioDraws('ASC_CAR_RND', 'NORMAL_ANTI')

ASC_TRANSIT = Beta('ASC_TRANSIT', 0, None, None, 0)
ASC_TRANSIT_S = Beta('ASC_TRANSIT_S', 1, None, None, 0)
ASC_TRANSIT_RND = ASC_TRANSIT + ASC_TRANSIT_S * bioDraws('ASC_TRANSIT_RND', 'NORMAL_ANTI')

ASC_RH = Beta('ASC_RH', 0, None, None, 0)
ASC_RH_S = Beta('ASC_RH_S', 1, None, None, 0)
ASC_RH_RND = ASC_RH + ASC_RH_S * bioDraws('ASC_RH_RND', 'NORMAL_ANTI')

ASC_SCOOTER = Beta('ASC_SCOOTER', 0, None, None, 0)
ASC_SCOOTER_S = Beta('ASC_SCOOTER_S', 1, None, None, 0)
ASC_SCOOTER_RND = ASC_SCOOTER + ASC_SCOOTER_S * bioDraws('ASC_SCOOTER_RND', 'NORMAL_ANTI')

ASC_DLBIKE = Beta('ASC_DLBIKE', 0, None, None, 0)
ASC_DLBIKE_S = Beta('ASC_DLBIKE_S', 1, None, None, 0)
ASC_DLBIKE_RND = ASC_DLBIKE + ASC_DLBIKE_S * bioDraws('ASC_DLBIKE_RND', 'NORMAL_ANTI')

ASC_DKBIKE = Beta('ASC_DKBIKE', 0, None, None, 0)
ASC_DKBIKE_S = Beta('ASC_DKBIKE_S', 1, None, None, 0)
ASC_DKBIKE_RND = ASC_DKBIKE + ASC_DKBIKE_S * bioDraws('ASC_DKBIKE_RND', 'NORMAL_ANTI')

ASC_SCTRANSIT = Beta('ASC_SCTRANSIT', 0, None, None, 0)
ASC_SCTRANSIT_S = Beta('ASC_SCTRANSIT_S', 1, None, None, 0)
ASC_SCTRANSIT_RND = ASC_SCTRANSIT + ASC_SCTRANSIT_S * bioDraws('ASC_SCTRANSIT_RND', 'NORMAL_ANTI')

ASC_WALK = Beta('ASC_WALK', 0, None, None, 0)
ASC_WALK_S = Beta('ASC_WALK_S', 1, None, None, 0 )
ASC_WALK_RND = ASC_WALK + ASC_WALK_S * bioDraws('ASC_WALK_RND', 'NORMAL_ANTI')

# BIKE mode parameters
ASC_BIKE = Beta('ASC_BIKE', 0, None, None, 0)
ASC_BIKE_S = Beta('ASC_BIKE_S', 1, None, None, 0)
ASC_BIKE_RND = ASC_BIKE + ASC_BIKE_S * bioDraws('ASC_BIKE_RND', 'NORMAL_ANTI')

# SCOOTER mode parameters
ASC_SCOOTER = Beta('ASC_SCOOTER', 0, None, None, 0)
ASC_SCOOTER_S = Beta('ASC_SCOOTER_S', 1, None, None, 0)
ASC_SCOOTER_RND = ASC_SCOOTER + ASC_SCOOTER_S * bioDraws('ASC_SCOOTER_RND', 'NORMAL_ANTI')

# DKBIKE mode parameters
ASC_DKBIKE = Beta('ASC_DKBIKE', 0, None, None, 0)
ASC_DKBIKE_S = Beta('ASC_DKBIKE_S', 1, None, None, 0)
ASC_DKBIKE_RND = ASC_DKBIKE + ASC_DKBIKE_S * bioDraws('ASC_DKBIKE_RND', 'NORMAL_ANTI')

# DLBIKE mode parameters
ASC_DLBIKE = Beta('ASC_DLBIKE', 0, None, None, 0)
ASC_DLBIKE_S = Beta('ASC_DLBIKE_S', 1, None, None, 0)
ASC_DLBIKE_RND = ASC_DLBIKE + ASC_DLBIKE_S * bioDraws('ASC_DLBIKE_RND', 'NORMAL_ANTI')

# SCTRANSIT mode parameters
ASC_SCTRANSIT = Beta('ASC_SCTRANSIT', 0, None, None, 0)
ASC_SCTRANSIT_S = Beta('ASC_SCTRANSIT_S', 1, None, None, 0)
ASC_SCTRANSIT_RND = ASC_SCTRANSIT + ASC_SCTRANSIT_S * bioDraws('ASC_SCTRANSIT_RND', 'NORMAL_ANTI')


In [17]:
# # Nested logit terms, groupings = micromobility, conventional, active transportation, other
# draw_micromobility = bioDraws('ASC_Micromobility', 'NORMAL_HALTON3')
# draw_activetransp = bioDraws('ASC_ActiveTransp', 'NORMAL_HALTON3')
# ASC_ACTIVETRANSP = Beta('ASC_ACTIVE TRANSP', 0, None, None, 0)
# ASC_MICROMOBILITY = Beta('ASC_MICROMOBILITY', 0, None, None, 0)


# ASC_WALK = Beta('ASC_WALK', 0, None, None, 0)
# ASC_WALK_S = Beta('ASC_WALK_S', 1, None, None, 0)
# ASC_WALK_RND = ASC_WALK + ASC_ACTIVETRANSP * draw_activetransp

# ASC_BIKE = Beta('ASC_BIKE', 0, None, None, 0)
# ASC_BIKE_S = Beta('ASC_BIKE_S', 1, None, None, 0)
# ASC_BIKE_RND = ASC_BIKE + ASC_ACTIVETRANSP * draw_activetransp

# ASC_SCOOTER = Beta('ASC_SCOOTER', 0, None, None, 0)
# ASC_SCOOTER_RND = ASC_SCOOTER + ASC_MICROMOBILITY * draw_micromobility + ASC_ACTIVETRANSP * draw_activetransp

# ASC_DLBIKE = Beta('ASC_DLBIKE', 0, None, None, 0)
# ASC_DLBIKE_RND = ASC_DLBIKE + ASC_MICROMOBILITY * draw_micromobility + ASC_ACTIVETRANSP * draw_activetransp 

# ASC_DKBIKE = Beta('ASC_DKBIKE', 0, None, None, 0)
# ASC_DKBIKE_RND = ASC_DKBIKE + ASC_MICROMOBILITY * draw_micromobility + ASC_ACTIVETRANSP * draw_activetransp

# ASC_SCTRANSIT = Beta('ASC_SCTRANSIT', 0, None, None, 0)
# ASC_SCTRANSIT_RND = ASC_SCTRANSIT + ASC_MICROMOBILITY * draw_micromobility + ASC_ACTIVETRANSP * draw_activetransp

In [18]:
B_ANTIMICRO_FA1_PERSBIKE = Beta('B_ANTIMICRO_FA1_PERSBIKE', 0, None, None, 0)
B_EXPECTEDPERF_INTENTTOUSE_PERSBIKE = Beta('B_EXPECTEDPERF_INTENTTOUSE_PERSBIKE', 0, None, None, 0)
B_SOCIALNORM_PERSBIKE = Beta('B_SOCIALNORM_PERSBIKE', 0, None, None, 0)
B_ENVVALUE_PERSBIKE = Beta('B_ENVVALUE_PERSBIKE', 0, None, None, 0)
B_ENVBENEFIT_PERSBIKE = Beta('B_ENVBENEFIT_PERSBIKE', 0, None, None, 0)
B_RESIDEFFECT_EXPECTEDPERF_PERSBIKE = Beta('B_RESIDEFFECT_EXPECTEDPERF_PERSBIKE', 0, None, None, 0)
B_RISK_PERSBIKE = Beta('B_RISK_PERSBIKE', 0, None, None, 0)
B_RESIDEFFECT_PERSBIKE = Beta('B_RESIDEFFECT_PERSBIKE', 0, None, None, 0)

B_ANTIMICRO_FA1_SCOOTER = Beta('B_ANTIMICRO_FA1_SCOOTER', 0, None, None, 0)
B_EXPECTEDPERF_INTENTTOUSE_SCOOTER = Beta('B_EXPECTEDPERF_INTENTTOUSE_SCOOTER', 0, None, None, 0)
B_SOCIALNORM_SCOOTER = Beta('B_SOCIALNORM_SCOOTER', 0, None, None, 0)
B_ENVVALUE_SCOOTER = Beta('B_ENVVALUE_SCOOTER', 0, None, None, 0)
B_ENVBENEFIT_SCOOTER = Beta('B_ENVBENEFIT_SCOOTER', 0, None, None, 0)
B_RESIDEFFECT_EXPECTEDPERF_SCOOTER = Beta('B_RESIDEFFECT_EXPECTEDPERF_SCOOTER', 0, None, None, 0)
B_RISK_SCOOTER = Beta('B_RISK_SCOOTER', 0, None, None, 0)
B_RESIDEFFECT_SCOOTER = Beta('B_RESIDEFFECT_SCOOTER', 0, None, None, 0)

B_ANTIMICRO_FA1_BIKE = Beta('B_ANTIMICRO_FA1_BIKE', 0, None, None, 0)
B_EXPECTEDPERF_INTENTTOUSE_BIKE = Beta('B_EXPECTEDPERF_INTENTTOUSE_BIKE', 0, None, None, 0)
B_SOCIALNORM_BIKE = Beta('B_SOCIALNORM_BIKE', 0, None, None, 0)
B_ENVVALUE_BIKE = Beta('B_ENVVALUE_BIKE', 0, None, None, 0)
B_ENVBENEFIT_BIKE = Beta('B_ENVBENEFIT_BIKE', 0, None, None, 0)
B_RESIDEFFECT_EXPECTEDPERF_BIKE = Beta('B_RESIDEFFECT_EXPECTEDPERF_BIKE', 0, None, None, 0)
B_RISK_BIKE = Beta('B_RISK_BIKE', 0, None, None, 0)
B_RESIDEFFECT_BIKE = Beta('B_RESIDEFFECT_BIKE', 0, None, None, 0)

B_ANTIMICRO_FA1_SCTRANSIT = Beta('B_ANTIMICRO_FA1_SCTRANSIT', 0, None, None, 0)
B_EXPECTEDPERF_INTENTTOUSE_SCTRANSIT = Beta('B_EXPECTEDPERF_INTENTTOUSE_SCTRANSIT', 0, None, None, 0)
B_SOCIALNORM_SCTRANSIT = Beta('B_SOCIALNORM_SCTRANSIT', 0, None, None, 0)
B_ENVVALUE_SCTRANSIT = Beta('B_ENVVALUE_SCTRANSIT', 0, None, None, 0)
B_ENVBENEFIT_SCTRANSIT = Beta('B_ENVBENEFIT_SCTRANSIT', 0, None, None, 0)
B_RESIDEFFECT_EXPECTEDPERF_SCTRANSIT = Beta('B_RESIDEFFECT_EXPECTEDPERF_SCTRANSIT', 0, None, None, 0)
B_RISK_SCTRANSIT = Beta('B_RISK_SCTRANSIT', 0, None, None, 0)
B_RESIDEFFECT_SCTRANSIT = Beta('B_RESIDEFFECT_SCTRANSIT', 0, None, None, 0)




In [19]:
B_CONV_NONHOMEBASE = Beta('B_CONV_NONHOMEBASE', 0, None, None, 0)
B_MICRO_NONHOMEBASE = Beta('B_MICRO_NONHOMEBASE', 0, None, None, 0)

In [22]:
# Utility functions
V1 = (ASC_CAR_RND
      + B_CARTIME*cartime
      )

V2 = (ASC_TRANSIT_RND
      + B_TRANSITTIME*transittime
      + B_CONV_NONHOMEBASE * trippurp_NHB
      )

V3 = (ASC_RH_RND
      + B_RHTIME*rdtime
      + B_CONV_NONHOMEBASE * trippurp_NHB
      )

V4 = (ASC_WALK_RND
      + B_WALKTIME*walktime
      + B_CONV_NONHOMEBASE * trippurp_NHB
      )

V5 = (ASC_BIKE_RND
      + B_PERSBIKETIME*biketime
      + B_CONV_NONHOMEBASE * trippurp_NHB
      + B_ANTIMICRO_FA1_PERSBIKE*Factor_1
      + B_EXPECTEDPERF_INTENTTOUSE_PERSBIKE*Factor_2
      + B_SOCIALNORM_PERSBIKE*Factor_3
      + B_ENVVALUE_PERSBIKE*Factor_4
      + B_ENVBENEFIT_PERSBIKE*Factor_5
      + B_RESIDEFFECT_EXPECTEDPERF_PERSBIKE*Factor_6
      + B_RISK_PERSBIKE*Factor_7
      + B_RESIDEFFECT_PERSBIKE*Factor_8

      )

V7 = (ASC_SCOOTER_RND
      + B_SCOOTERTIME*sctime
      # + B_SCOOTERTIME_PRIME*sctime*AVtech
      + B_COST*sccost_adj
      # + B_COST_PRIME*sccost_adj*AVtech
      + B_ACCESS*SCAW*(1-AVtech)
      + B_WAITAV*SCAV*AVtech
      + B_MICRO_NONHOMEBASE * trippurp_NHB
      + B_ANTIMICRO_FA1_SCOOTER*Factor_1
      + B_EXPECTEDPERF_INTENTTOUSE_SCOOTER*Factor_2
      + B_SOCIALNORM_SCOOTER*Factor_3
      + B_ENVVALUE_SCOOTER*Factor_4
      + B_ENVBENEFIT_SCOOTER*Factor_5
      + B_RESIDEFFECT_EXPECTEDPERF_SCOOTER*Factor_6
      + B_RISK_SCOOTER*Factor_7
      + B_RESIDEFFECT_SCOOTER*Factor_8

      )
V8 = (ASC_DLBIKE_RND
      + B_BIKETIME*dltime
      + B_COST*dlcost_adj
      + B_ACCESS*DLAW
      + B_MICRO_NONHOMEBASE * trippurp_NHB
      + B_ANTIMICRO_FA1_BIKE*Factor_1
      + B_EXPECTEDPERF_INTENTTOUSE_BIKE*Factor_2
      + B_SOCIALNORM_BIKE*Factor_3
      + B_ENVVALUE_BIKE*Factor_4
      + B_ENVBENEFIT_BIKE*Factor_5
      + B_RESIDEFFECT_EXPECTEDPERF_BIKE*Factor_6
      + B_RISK_BIKE*Factor_7
      + B_RESIDEFFECT_BIKE*Factor_8

      )
V9 = (ASC_DKBIKE_RND
      + B_BIKETIME*dbtime
      + B_COST*dbcost_adj
      + B_ACCESS*DBAW
      + B_DROP*DBDW
      + B_MICRO_NONHOMEBASE * trippurp_NHB
      + B_ANTIMICRO_FA1_BIKE*Factor_1
      + B_EXPECTEDPERF_INTENTTOUSE_BIKE*Factor_2
      + B_SOCIALNORM_BIKE*Factor_3
      + B_ENVVALUE_BIKE*Factor_4
      + B_ENVBENEFIT_BIKE*Factor_5
      + B_RESIDEFFECT_EXPECTEDPERF_BIKE*Factor_6
      + B_RISK_BIKE*Factor_7
      + B_RESIDEFFECT_BIKE*Factor_8


      )
V10 = (ASC_SCTRANSIT_RND
       + B_SCTRANSITTIME*sttotime
       + B_COST*sttocost_adj
       + B_ACCESS*STAW*(1-AVtech)
       + B_WAITAV*STAV*AVtech
       + B_MICRO_NONHOMEBASE * trippurp_NHB
       + B_ANTIMICRO_FA1_SCTRANSIT*Factor_1
       + B_EXPECTEDPERF_INTENTTOUSE_SCTRANSIT*Factor_2
       + B_SOCIALNORM_SCTRANSIT*Factor_3
       + B_ENVVALUE_SCTRANSIT*Factor_4
       + B_ENVBENEFIT_SCTRANSIT*Factor_5
       + B_RESIDEFFECT_EXPECTEDPERF_SCTRANSIT*Factor_6
       + B_RISK_SCTRANSIT*Factor_7
       + B_RESIDEFFECT_SCTRANSIT*Factor_8

       )


In [23]:
V = {1: V1, 2: V2, 3: V3, 4: V4, 5: V5, 7: V7, 8: V8, 9: V9, 10: V10}
av = {1: car_av, 2: transit_av, 3: rd_av, 4: walk_av, 5: bike_av, 
      7: scooter_av, 8: dlbike_av, 9: dkbike_av, 10: sctransit_av}
      

condprob = models.logit(V, av, choice)
condlikeindiv = PanelLikelihoodTrajectory(condprob)
logprob = log(MonteCarlo(condlikeindiv))

biogeme = bio.BIOGEME(database, logprob, numberOfDraws = 50, numberOfThreads = 100)

# biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = "Mixed Logit with Social Norm, 6 indicators - 250 draws"
# biogeme.modelName = "HybridChoiceNestedLogit"

results = biogeme.estimate()
pandasResults = results.getEstimatedParameters()
print(f"Estimated betas: {len(results.data.betaValues)}")
print(f"Final log likelihood: {results.data.logLike:.3f}")
print(pandasResults.round(2))

The use of argument numberOfThreads in the constructor of the BIOGEME object is deprecated and will be removed in future versions of Biogeme. Instead, define parameter number_of_threads in section MultiThreadingof the .toml parameter file. The default file name is biogeme.toml
The use of argument numberOfDraws in the constructor of the BIOGEME object is deprecated and will be removed in future versions of Biogeme. Instead, define parameter number_of_draws in section MonteCarloof the .toml parameter file. The default file name is biogeme.toml


In [ ]:
# corr = calculate_correlation(
#     nests, results, alternative_names = {1: 'Car', 2: 'Transit', 3: 'Ridehailing', 4: 'Walk', 5: 'Personal Bike',
#                                          7: 'Scooter', 8: 'Dockless Bike', 9: 'Docked Bike', 10: 'Scooter + Transit'}
# )
# print(corr)